In [0]:
import os
os.getcwd()

'/content'

In [0]:
os.listdir(os.getcwd())

['sample_data', '.config']

In [0]:
os.mkdir('Crop2')

In [0]:
os.listdir('/content/Crop2')

[]

In [0]:
os.chdir('Crop2')

In [0]:
os.getcwd()

'/content/Crop2'

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print('Saved file', name)

Saved file Crop_images.zip


In [0]:
os.listdir('/content/Crop1')

['Crop_images.zip']

In [0]:
# This is for extracting the files
import zipfile
path_to_zip_file = 'Crop_images.zip'
directory_to_extract_to = ''
zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
zip_ref.extractall(directory_to_extract_to)
zip_ref.close()

In [0]:
os.listdir('/content/Crop1')

['Crop images', 'Crop_images.zip']

In [0]:
os.listdir('/content/Crop1/Crop images')

['Traning', 'Testing']

In [0]:
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf

In [0]:
!ls

'Crop images'   Crop_images.zip


In [0]:
train_data_dir = '/content/Crop1/Crop images/Traning'  
validation_data_dir = '/content/Crop1/Crop images/Testing'

In [0]:
target_names = [item for item in os.listdir(train_data_dir) if os.path.isdir(os.path.join(train_data_dir, item))]
nb_train_samples = sum([len(files) for _, _, files in os.walk(train_data_dir)])  
nb_validation_samples = sum([len(files) for _, _, files in os.walk(validation_data_dir)])
total_nb_samples = nb_train_samples + nb_validation_samples 

nb_classes = len(target_names)      # number of output classes

In [0]:
batch_size = 128
epochs = 12

print('Training a CNN Multi-Classifier Model ......')
print('\n - names of classes: ', target_names, '\n - # of classes: ', nb_classes)
print(' - # of trained samples: ', nb_train_samples, '\n - # of validation samples: ', nb_validation_samples,
      '\n - total # of samples: ', total_nb_samples, '\n - train ratio:', round(nb_train_samples/total_nb_samples*100, 2),
      '\n - validation ratio:', round(nb_validation_samples/total_nb_samples*100, 2),
      ' %', '\n - # of epochs: ', epochs, '\n - batch size: ', batch_size)

Training a CNN Multi-Classifier Model ......

 - names of classes:  ['brown spot', 'blast', 'sheath blight', 'blb'] 
 - # of classes:  4
 - # of trained samples:  131 
 - # of validation samples:  53 
 - total # of samples:  184 
 - train ratio: 71.2 
 - validation ratio: 28.8  % 
 - # of epochs:  12 
 - batch size:  128


In [0]:
import sys 
import os
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import keras
from keras.callbacks import TensorBoard
from keras.applications.mobilenet import MobileNet

# fix seed for reproducible results (only works on CPU, not GPU)
seed = 9
np.random.seed(seed=seed)
tf.set_random_seed(seed=seed)

Using TensorFlow backend.


In [0]:
# All defined hyper parameters for model
nb_classes = 4                             # number of classes
based_model_last_block_layer_number = 126  # value is based on based model selected.
img_width, img_height = 150, 150           # change based on the shape/structure of your images for squeezenet
batch_size = 32                            # dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 5                              # number of iteration the algorithm gets trained.
learn_rate = 1e-4                          # sgd learning rate
momentum = .9                              # sgd momentum to avoid local minimum
transformation_ratio = .05                 # how aggressive will be the data augmentation/transformation
nb_train_samples = 131                     # Total number of train samples. NOT including augmented images
nb_validation_samples = 53                 # Total number of train samples. NOT including augmented image

In [0]:
def train(train_data_dir, validation_data_dir, model_path):
    # Pre-Trained CNN Model using imagenet dataset for pre-trained weights
       
    if K.image_data_format() == 'channels_first': 
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)
    
    # We use Xception as it gives more stable performance
    base_model = ResNet50(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)  
    #base_model = Xception(input_shape=input_shape, weights='imagenet', include_top=False)
    #base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_width, img_height, 3))
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)

    predictions = Dense(nb_classes, activation='softmax', name='predictions')(x)

    # add the top layer block to your base model
    model = Model(base_model.input, predictions)
    print(model.summary())

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all layers of the based model that is already pre-trained.
    for layer in base_model.layers:
        layer.trainable = False

    # Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.
    # To save augmentations un-comment save lines and add to your flow parameters.
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=transformation_ratio,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True,
                                       vertical_flip=True)

    validation_datagen = ImageDataGenerator(rescale=1. / 255)
    # os.makedirs(os.path.join(os.path.abspath(train_data_dir), 'preview'), exist_ok=True)
    train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size=[img_width, img_height],
                                                        batch_size=batch_size,
                                                        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                                  target_size=[img_width, img_height],
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical')

    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])

    # save weights of best training epoch: monitor either val_loss or val_acc

    top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
    callbacks_list = [
        ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        #EarlyStopping(monitor='val_acc', patience=5, verbose=0),
        keras.callbacks.TensorBoard(log_dir='tensorboard/inception-v3-train-top-layer', histogram_freq=0, write_graph=False, write_images=False)
    ]

    # Train Simple CNN
    model.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=nb_epoch / 5,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size,
                        callbacks=callbacks_list)

    # verbose
    print("\nStarting to Fine Tune Model\n")
    # add the best weights from the train top model
    # at this point we have the pre-train weights of the base model and the trained weight of the new/added top model
    # we re-load model weights to ensure the best epoch is selected and not the last one.
    model.load_weights(top_weights_path)

    for layer in model.layers[:based_model_last_block_layer_number]:
        layer.trainable = False
    for layer in model.layers[based_model_last_block_layer_number:]:
        layer.trainable = True

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
                  
    
    # save weights of best training epoch: monitor either val_loss or val_acc
    final_weights_path = os.path.join(os.path.abspath(model_path), 'model_weights.h5')
    callbacks_list = [
        ModelCheckpoint(final_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5, verbose=0),
        keras.callbacks.TensorBoard(log_dir='tensorboard/inception-v3-fine-tune', histogram_freq=0, write_graph=False, write_images=False)
    ]
    
 
    newpath = '/content/Crop1'    # Path for storing tensorboard file 
    tensorboard = TensorBoard(log_dir = newpath)


    # fine-tune the model
    model.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=nb_epoch,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size,
                        callbacks=[TensorBoardColabCallback(tbc)])

    # save model
    model_json = model.to_json()
    with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
        json_file.write(model_json)

# Path for Train & validation dataset. Give this folder path to run again 

train(r'/content/Crop1/Crop images/Traning', r'/content/Crop1/Crop images/Testing', '.')

219062272/219055592 [==============================] - 2s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_115 (Conv2D)             (None, 74, 74, 32)   864         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_115 (BatchN (None, 74, 74, 32)   96          conv2d_115[0][0]                 
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 74, 74, 32)   0           batch_normalization_115[0][0]    
__________________________________________

In [0]:
!rm -rf <folder_name>

In [0]:
!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://0a5b0edf.ngrok.io
